# Day 22: Grid Computing

## Part One

You gain access to a massive storage cluster arranged in a grid; each storage node is only connected to the four nodes directly adjacent to it (three if the node is on an edge, two if it's in a corner).

You can directly access data only on node `/dev/grid/node-x0-y0`, but you can perform some limited actions on the other nodes:

* You can get the disk usage of all nodes (via df). The result of doing this is in your puzzle input.

* You can instruct a node to move (not copy) all of its data to an adjacent node (if the destination node has enough space to receive the data). The sending node is left empty after this operation.

Nodes are named by their position: the node named `node-x10-y10` is adjacent to nodes `node-x9-y10`, `node-x11-y10`, `node-x10-y9`, and `node-x10-y11`.

Before you begin, you need to understand the arrangement of data on these nodes. Even though you can only move data between directly connected nodes, you're going to need to rearrange a lot of the data to get access to the data you need. Therefore, you need to work out how you might be able to shift data around.

To do this, you'd like to count the number of viable pairs of nodes. A viable pair is any two nodes `(A,B)`, regardless of whether they are directly connected, such that:

* Node A is not empty (its Used is not zero).

* Nodes A and B are not the same node.

* The data on node A (its `Used`) would fit on node B (its `Avail`).

How many viable pairs of nodes are there?

---

In [223]:
import re

re_df = r'/dev/grid/node-x(\d+)-y(\d+)\s+(\d+)T\s+(\d+)T\s+(\d+)T\s+(\d+)\%'

# inputs[(x,y)] = (size, used, avail, use%)
inputs = {
    (int(x[1]), int(x[2])) : (int(x[3]), int(x[4]), int(x[5]), int(x[6])) for i in open('Day22.in').readlines() if (x:=re.match(re_df, i))
}

In [224]:
ab_pairs = 0

for v in inputs:
    for u in inputs:
        if v != u:
            if inputs[v][1] != 0:
                if inputs[v][1] <= inputs[u][2]:
                    ab_pairs += 1

In [225]:
ab_pairs

937

---

## Part Two

Now that you have a better understanding of the grid, it's time to get to work.

Your goal is to gain access to the data which begins in the node with `y=0` and the highest `x` (that is, the node in the top-right corner).

For example, suppose you have the following grid:

```
Filesystem            Size  Used  Avail  Use%
/dev/grid/node-x0-y0   10T    8T     2T   80%
/dev/grid/node-x0-y1   11T    6T     5T   54%
/dev/grid/node-x0-y2   32T   28T     4T   87%
/dev/grid/node-x1-y0    9T    7T     2T   77%
/dev/grid/node-x1-y1    8T    0T     8T    0%
/dev/grid/node-x1-y2   11T    7T     4T   63%
/dev/grid/node-x2-y0   10T    6T     4T   60%
/dev/grid/node-x2-y1    9T    8T     1T   88%
/dev/grid/node-x2-y2    9T    6T     3T   66%
```

In this example, you have a storage grid `3` nodes wide and `3` nodes tall. The node you can access directly, `node-x0-y0`, is almost full. The node containing the data you want to access, `node-x2-y0` (because it has `y=0` and the highest `x` value), contains `6` terabytes of data - enough to fit on your node, if only you could make enough space to move it there.

Fortunately, `node-x1-y1` looks like it has enough free space to enable you to move some of this data around. In fact, it seems like all of the nodes have enough space to hold any node's data (except `node-x0-y2`, which is much larger, very full, and not moving any time soon). So, initially, the grid's capacities and connections look like this:

```
( 8T/10T) --  7T/ 9T -- [ 6T/10T]
    |           |           |
  6T/11T  --  0T/ 8T --   8T/ 9T
    |           |           |
 28T/32T  --  7T/11T --   6T/ 9T
```

The node you can access directly is in parentheses; the data you want starts in the node marked by square brackets.

In this example, most of the nodes are interchangable: they're full enough that no other node's data would fit, but small enough that their data could be moved around. Let's draw these nodes as `.`. The exceptions are the empty node, which we'll draw as `_`, and the very large, very full node, which we'll draw as `#`. Let's also draw the goal data as `G`. Then, it looks like this:
```
(.) .  G
 .  _  .
 #  .  .
```
 
The goal is to move the data in the top right, `G`, to the node in parentheses. To do this, we can issue some commands to the grid and rearrange the data:

Move data from `node-y0-x1` to `node-y1-x1`, leaving node `node-y0-x1` empty:

```
(.) _  G
 .  .  .
 #  .  .
```

Move the goal data from `node-y0-x2` to `node-y0-x1`:

```
(.) G  _
 .  .  .
 #  .  .
```

At this point, we're quite close. However, we have no deletion command, so we have to move some more data around. So, next, we move the data from `node-y1-x2` to `node-y0-x2`:

```
(.) G  .
 .  .  _
 #  .  .
```

Move the data from `node-y1-x1` to `node-y1-x2`:

```
(.) G  .
 .  _  .
 #  .  .
```

Move the data from `node-y1-x0` to `node-y1-x1`:

```
(.) G  .
 _  .  .
 #  .  .
```

Next, we can free up space on our node by moving the data from `node-y0-x0` to `node-y1-x0`:

```
(_) G  .
 .  .  .
 #  .  .
```

Finally, we can access the goal data by moving the it from `node-y0-x1` to `node-y0-x0`:

```
(G) _  .
 .  .  .
 #  .  .
```

So, after `7` steps, we've accessed the data we want. Unfortunately, each of these moves takes time, and we need to be efficient:

What is the fewest number of steps required to move your goal data to `node-x0-y0`?

---

In [226]:
grid = {
    i: '_' if inputs[i][1] == 0 else ('.' if inputs[i][1] < 90 else '#') for i in inputs
}

In [227]:
rows, cols = max([x for x, y in grid]), max([y for x, y in grid])

s = ' '

for x in range(rows+1):
    for y in range(cols+1):
        if x == y == 0:
            s = s[:-1]
            s += f'({grid[(x, y)]})'
        elif (x == rows) & (y == 0):
            s = s[:-1]
            s += f'[{grid[(x, y)]}]'
        else:
            s += grid[(x, y)] + ' '
    s += '\n\n '

In [228]:
print(s)

(.). . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . # . . . . . . . _ . . . . . . . . . 

 . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . .

A state can be thought of as a double (b, d) that complies with the map -- d for data and b for blank.

b can be moved up, down, left, right and interacts with d by swapping its place. It is a valid move if b is not moving onto a hash on the map.

In [229]:
blanks = [i for i in grid if grid[i] == '_']
states = set([(i, (max([x for x, y in grid]), 0)) for i in blanks])

current_states = states.copy()

def legal_moves(blank, grid = grid):
    nodes = []
    x, y = blank
    
    for i in range(-1, 2, 2):
        if ((x+i, y) in grid) and (grid[(x+i, y)] != '#'):
            nodes.append((x+i, y))
        if ((x, y+i) in grid) and (grid[(x, y+i)] != '#'):
            nodes.append((x, y+i))
    
    return nodes

def get_new_states(current_states, states=states, grid=grid):
    
    new_states = set()
    
    for b, d in current_states:
        legals = legal_moves(b, grid)
        for legal in legals:
            if legal == d:
                if (legal, b) not in states:
                    new_states.add((legal, b))
            else:
                if (legal, d) not in states:
                    new_states.add((legal, d))
    
    return new_states


cost = 0

while (((0,1), (0,0)) not in current_states) and (((1,0), (0,0)) not in current_states):
    current_states = get_new_states(current_states)
    states |= current_states
    cost += 1


In [230]:
cost

188

In [144]:
current_states

{((0, 29), (0, 31)), ((0, 31), (0, 30)), ((1, 30), (0, 31))}

In [189]:
test = '''/dev/grid/node-x0-y0   10T    8T     2T   80%
/dev/grid/node-x0-y1   11T    6T     5T   54%
/dev/grid/node-x0-y2   32T   28T     4T   87%
/dev/grid/node-x1-y0    9T    7T     2T   77%
/dev/grid/node-x1-y1    8T    0T     8T    0%
/dev/grid/node-x1-y2   11T    7T     4T   63%
/dev/grid/node-x2-y0   10T    6T     4T   60%
/dev/grid/node-x2-y1    9T    8T     1T   88%
/dev/grid/node-x2-y2    9T    6T     3T   66%'''

inputs = {
    (int(x[1]), int(x[2])) : (int(x[3]), int(x[4]), int(x[5]), int(x[6])) for i in test.split('\n') if (x:=re.match(re_df, i))
}

grid = {
    i: '_' if inputs[i][1] == 0 else ('.' if inputs[i][1] < 10 else '#') for i in inputs
}



In [196]:
rows, cols = max([x for x, y in grid]), max([y for x, y in grid])

s = ' '

for y in range(cols+1):
    for x in range(rows+1):
        if x == y == 0:
            s = s[:-1]
            s += f'({grid[(x, y)]})'
        elif (x == rows) & (y == 0):
            s = s[:-1]
            s += f'[{grid[(x, y)]}]'
        else:
            s += grid[(x, y)] + ' '
    s += '\n\n '

In [197]:
print(s)

(.).[.]

 . _ . 

 # . . 

 


In [219]:
blanks = [i for i in grid if grid[i] == '_']
states = set([(i, (max([x for x, y in grid]), 0)) for i in blanks])

current_states = states.copy()
cost = 0

while (((0,1), (0,0)) not in current_states) and (((1,0), (0,0)) not in current_states):
    current_states = get_new_states(current_states, grid)
    states |= current_states
    cost += 1

In [220]:
cost

7

In [209]:
blanks = [i for i in grid if grid[i] == '_']
states = set([(i, (max([x for x, y in grid]), 0)) for i in blanks])

current_states = states.copy()

In [204]:
states = set([(i, (max([x for x, y in grid]), 0)) for i in blanks])
